In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join('..', 'src')))
from funkwpap import *
import sympy as sp, pandas as pd, numpy as np, tqdm, time, sys, matplotlib.pyplot as plt, tqdm
from statistics import mean
from scipy.optimize import fsolve
import gurobipy as gb


In [2]:
import sympy as sp

# Define symbolic variable
x = sp.symbols('x')

# Create a Regulator with an emission cap
RegulatorETS = Regulator("ETS", permit_price=20.0, emission_cap=1000000)

# Define Countries with updated names and attributes
country1 = Country('Atlantis', size=50, regulator=RegulatorETS, GDP_per_capita=400, population=100000000, industry_percentage=30)


# Define Sectors with demand functions
sector1 = Sector('Steel', price_demand_function=200 - 0.1*x, free_emission_multiplier=0.8, regulator=RegulatorETS)
sector2 = Sector('Cement', price_demand_function=150 - 0.05*x, free_emission_multiplier=0.85, regulator=RegulatorETS)


# Create Firms with updated definitions (production_cost_function=0, abatement_cost_function as previous production_cost_function)
# Firms in Sector 1: Steel
firm_S1_F1 = Firm('S1_F1', sector1, country1, x*0, 2*x + 3*x**2 + x**3, 10000, 5000, 0, RegulatorETS)
# firm_S1_F2 = Firm('S1_F2', sector1, country1, x*0, 3*x + 2*x**2 + 2*x**3, 12000, 6000, 0, RegulatorETS)


# # Firms in Sector 2: Cement
# firm_S2_F1 = Firm('S2_F1', sector2, country1, x*0, x + 2*x**2 + 3*x**3, 9000, 4500, 0, RegulatorETS)
firm_S2_F2 = Firm('S2_F2', sector2, country1, x*0, 2*x + x**2 + 2*x**3, 9500, 4750, 0, RegulatorETS)

In [3]:
RegulatorETS.BAU_calculator(precision = 0.01, print_diff = True)
#print(RegulatorETS.BAU_emissions)
# 17638.87773  3750 for 2
RegulatorETS.emission_cap = RegulatorETS.BAU_emissions*0.8
# Optimal solution found for 2 sectors.
# Firm S1_F1 has output 451.62827788960453 and emission 446.7349764049005
# Firm S1_F2 has output 451.6282778896044 and emission 447.8616178523027
# Firm S1_F3 has output 451.6282778896045 and emission 448.41767394370135
# Firm S2_F1 has output 556.5131115584181 and emission 553.3583289997191
# Firm S2_F2 has output 556.513111558418 and emission 552.5696348002654
# Firm S2_F3 has output 556.5131115584182 and emission 551.0577679991109
# Permit price: 103.19300716884365

Set parameter Username
Academic license - for non-commercial use only - expires 2025-08-14
Max diff: 0.00, permit price = 20.0, cap = 1000000, second_stage = False, a = 1.00.00

In [4]:
RegulatorETS.optimization_with_positive_constraints_ab(print_output= True, gurobi_print=False)
RegulatorETS.equilibrium_tester(full_output=True)

Optimal solution found
Firm S1_F1 has output 805.0355980200349 and emission 797.953428303229
Firm S2_F2 has output 1207.553397030052 and emission 1202.046571696771
Permit price: 194.96440197996523
Firm       | FOC 1        | FOC 2         | SOC 1         | SOC 2        | Hessian      | Status
-------------------------------------------------------------------------------------
Worst      |  +1.94e-7 ✔️ |  +1.94e-7 ✔️ |  -48.4930 ✔️ |  -48.6930 ✔️ |   +6.8082 ✔️ | 5/5
S1_F1      |  -1.38e-8 ✔️ |  +1.38e-8 ✔️ |  -48.4930 ✔️ |  -48.6930 ✔️ |   +9.6986 ✔️ | 5/5
S2_F2      |  -1.94e-7 ✔️ |  +1.94e-7 ✔️ |  -68.0819 ✔️ |  -68.1819 ✔️ |   +6.8082 ✔️ | 5/5


(True, 1.94192807612126e-7)

In [5]:
coeff = RegulatorETS.exp7_find_coefficient_for_free_allocation(print_output=True, percentage=0.4)

Lower bound: 0, Upper bound: 1.0, Current bound: 0.5, Total free allocation: 1670.448865891572
Lower bound: 0, Upper bound: 0.5, Current bound: 0.25, Total free allocation: 831.8780917896597
Lower bound: 0.25, Upper bound: 0.5, Current bound: 0.375, Total free allocation: 415.68448331503976
Lower bound: 0.375, Upper bound: 0.5, Current bound: 0.4375, Total free allocation: 623.6949946103666
Lower bound: 0.4375, Upper bound: 0.5, Current bound: 0.46875, Total free allocation: 727.7607499767088
Lower bound: 0.46875, Upper bound: 0.5, Current bound: 0.484375, Total free allocation: 779.8123086789137
Lower bound: 0.46875, Upper bound: 0.484375, Current bound: 0.4765625, Total free allocation: 805.8433281591517
Lower bound: 0.4765625, Upper bound: 0.484375, Current bound: 0.48046875, Total free allocation: 792.8273627085894


In [6]:
for sector in RegulatorETS.sector_registry.values():
    sector.free_emission_multiplier *= coeff

In [7]:
RegulatorETS.optimization_with_positive_constraints_ab(print_output= True, gurobi_print=False)
for firm in RegulatorETS.firm_registry.values():
    firm.calculate_profit_components()
print(sum([firm.free_permits for firm in RegulatorETS.firm_registry.values()]))
print(RegulatorETS.emission_cap*0.4)

Optimal solution found
Firm S1_F1 has output 830.9777295967303 and emission 827.6606785818444
Firm S2_F2 has output 1175.1469181145278 and emission 1172.3393214181556
Permit price: 54.91078835436175
799.3352300046569
800.0


In [8]:
consumer_surplus = 0
for sector in RegulatorETS.sector_registry.values():
    consumer_surplus += sector.get_consumer_surplus()
print(consumer_surplus)
# 721857.091272771

69050.4563331386


In [9]:
for sector in RegulatorETS.sector_registry.values():
    print(f"{sector.name} has a free emission multiplier {sector.free_emission_multiplier}")

Steel has a free emission multiplier 0.384375
Cement has a free emission multiplier 0.4083984375


In [10]:
for firm in RegulatorETS.firm_registry.values():
    firm.calculate_profit_components()

In [11]:
# =======================
# 1. Configuration Variables
# =======================
RUN_NAME = 'run_ULM'  # Unique identifier for the current run
BASE_PATH = 'C:/Users/Kostas/Documents/GitHub/Diplomatiki_kwpap_step_1/2024 Project v2/data/exp7'  # Fixed base path for data storage

# =======================
# 5. Data Saving
# =======================

def save_data(regulator, run_name, base_path):
    """
    Saves the data from the regulator object into CSV files.

    Parameters:
    - regulator (Regulator): The regulator instance containing all data.
    - run_name (str): Unique identifier for the current run.
    - base_path (str): Fixed base directory for data storage.
    """
    # Define run-specific paths
    run_path = os.path.join(base_path, run_name)
    data_dir = os.path.join(run_path, 'data')
    summaries_dir = os.path.join(run_path, 'summaries')

    # Create directories if they don't exist
    os.makedirs(data_dir, exist_ok=True)
    os.makedirs(summaries_dir, exist_ok=True)

    # 1. Extract Regulator Data
    regulator_data = {
        'id': [regulator.id],
        'name': [regulator.name],
        'permit_price': [regulator.permit_price],
        'emission_cap': [regulator.emission_cap],
        'BAU_emissions': [regulator.BAU_emissions],
        'BAU_profit': [regulator.BAU_profit]
    }

    df_regulator = pd.DataFrame(regulator_data)

    # Save Regulator DataFrame to CSV
    regulator_csv = os.path.join(data_dir, f'{run_name}_regulator.csv')
    df_regulator.to_csv(regulator_csv, index=False)
    print(f"Regulator data saved to {regulator_csv}.")

    # 2. Extract Countries Data
    countries = regulator.country_registry.values()
    country_list = []
    for country in countries:
        country_list.append({
            'id': country.id,
            'name': country.name,
            'size': country.size,
            'GDP_per_capita': country.GDP_per_capita,
            'population': country.population,
            'industry_percentage': country.industry_percentage,
            'GDP': country.GDP
        })

    df_countries = pd.DataFrame(country_list)

    # Save Countries DataFrame to CSV
    countries_csv = os.path.join(data_dir, f'{run_name}_countries.csv')
    df_countries.to_csv(countries_csv, index=False)
    print(f"Countries data saved to {countries_csv}.")

    # 3. Extract Sectors Data
    sectors = regulator.sector_registry.values()
    sector_list = []
    for sector in sectors:
        sector_list.append({
            'id': sector.id,
            'name': sector.name,
            'price_demand_function': str(sector.price_demand_function),
            'free_emission_multiplier': sector.free_emission_multiplier,
            'consumer_surplus': sector.consumer_surplus  # Include consumer_surplus
        })

    df_sectors = pd.DataFrame(sector_list)

    # Save Sectors DataFrame to CSV
    sectors_csv = os.path.join(data_dir, f'{run_name}_sectors.csv')
    df_sectors.to_csv(sectors_csv, index=False)
    print(f"Sectors data saved to {sectors_csv}.")

    # 4. Extract Firms Data
    firm_list = []
    for sector in sectors:
        for firm in sector.firms:
            firm_list.append({
                'id': firm.id,
                'name': firm.name,
                'sector_id': firm.sector.id,
                'country_id': firm.country.id,
                'production_cost_function': str(firm.production_cost_function),
                'abatement_cost_function': str(firm.abatement_cost_function),
                'actual_output': firm.actual_output,
                'emission': firm.emission,
                'profit': firm.profit,
                'BAU_output': firm.BAU_output,
                'BAU_emission': firm.BAU_emission,
                'BAU_profit': firm.BAU_profit,
                'exp7_free_allocation': firm.exp7_free_allocation,
                'abatement': firm.abatement,
                'permits_costs': firm.permits_costs,
                'sales': firm.sales,
                'permits_used': firm.permits_used,
                'free_permits': firm.free_permits,
                'permits_bought': firm.permits_bought
            })

    df_firms = pd.DataFrame(firm_list)

    # Save Firms DataFrame to CSV
    firms_csv = os.path.join(data_dir, f'{run_name}_firms.csv')
    df_firms.to_csv(firms_csv, index=False)
    print(f"Firms data saved to {firms_csv}.")

    # 5. Save Summary Statistics
    # Regulator Summary
    regulator_summary_csv = os.path.join(summaries_dir, f'{run_name}_summary_regulator.csv')
    df_regulator.describe().to_csv(regulator_summary_csv)
    print(f"Regulator summary saved to {regulator_summary_csv}.")

    # Sectors Summary
    sectors_summary_csv = os.path.join(summaries_dir, f'{run_name}_summary_sectors.csv')
    df_sectors.describe(include='all').to_csv(sectors_summary_csv)
    print(f"Sectors summary saved to {sectors_summary_csv}.")

    # Countries Summary
    countries_summary_csv = os.path.join(summaries_dir, f'{run_name}_summary_countries.csv')
    df_countries.describe().to_csv(countries_summary_csv)
    print(f"Countries summary saved to {countries_summary_csv}.")

    # Firms Summary
    firms_summary_csv = os.path.join(summaries_dir, f'{run_name}_summary_firms.csv')
    df_firms.describe(include='all').to_csv(firms_summary_csv)
    print(f"Firms summary saved to {firms_summary_csv}.")

    print("\nAll data and summaries have been saved successfully.")

# =======================
# 6. Execute Data Saving
# =======================
save_data(RegulatorETS, RUN_NAME, BASE_PATH)

Regulator data saved to C:/Users/Kostas/Documents/GitHub/Diplomatiki_kwpap_step_1/2024 Project v2/data/exp7\run_ULM\data\run_ULM_regulator.csv.
Countries data saved to C:/Users/Kostas/Documents/GitHub/Diplomatiki_kwpap_step_1/2024 Project v2/data/exp7\run_ULM\data\run_ULM_countries.csv.
Sectors data saved to C:/Users/Kostas/Documents/GitHub/Diplomatiki_kwpap_step_1/2024 Project v2/data/exp7\run_ULM\data\run_ULM_sectors.csv.


PermissionError: [Errno 13] Permission denied: 'C:/Users/Kostas/Documents/GitHub/Diplomatiki_kwpap_step_1/2024 Project v2/data/exp7\\run_ULM\\data\\run_ULM_firms.csv'

In [ ]:
import optuna

# Define the objective function for optimization
def objective(trial):
    # Define possible multiplier values from 0 to 1 with 0.05 increments
    multiplier_values = [i * 0.05 for i in range(21)]
    
    # Assign a multiplier to each sector using sector names as keys
    multipliers = {}
    for sector in RegulatorETS.sector_registry.values():
        multipliers[sector.name] = trial.suggest_categorical(
            f"multiplier_{sector.name}", multiplier_values
        )
    
    try:
        # Set the free_emission_multiplier for each sector
        for sector in RegulatorETS.sector_registry.values():
            sector.free_emission_multiplier = multipliers[sector.name]
        
        # Find the scaling coefficient to maintain the emission cap percentage
        coeff = RegulatorETS.exp7_find_coefficient_for_free_allocation(
            print_output=False, percentage=0.4
        )
        
        # Check if coefficient was found
        if coeff is None:
            raise ValueError("Scaling coefficient not found.")
        
        # Scale the free_emission_multiplier for each sector
        for sector in RegulatorETS.sector_registry.values():
            sector.free_emission_multiplier *= coeff
        
        # Run the economic simulation with the updated multipliers
        RegulatorETS.optimization_with_positive_constraints_ab(
            print_output=False, gurobi_print=False
        )
        if (any (sector.free_emission_multiplier > 1 for sector in RegulatorETS.sector_registry.values())):
            return float('inf')  # Penalize invalid configurations
        
        # Calculate the total consumer surplus
        consumer_surplus = sum(
            sector.get_consumer_surplus() for sector in RegulatorETS.sector_registry.values()
        )
        
        # Since Optuna minimizes the objective, return the negative consumer surplus
        return -consumer_surplus
    
    except KeyError as e:
        print(f"KeyError encountered: {e}")
        return float('inf')  # Penalize invalid configurations
    except Exception as e:
        print(f"An error occurred: {e}")
        return float('inf')  # Penalize invalid configurations

# Create an Optuna study to maximize consumer surplus
study = optuna.create_study(direction='minimize')

# Optimize the objective function
study.optimize(objective, n_trials=100, show_progress_bar=True)

# Retrieve the best multipliers found
best_multipliers = {
    sector_name.replace("multiplier_", ""): value
    for sector_name, value in study.best_params.items()
}

# Display the optimal free_emission_multipliers for each sector
print("Optimal free_emission_multipliers:")
for sector, multiplier in best_multipliers.items():
    print(f"{sector}: {multiplier}")

# Optionally, set the best multipliers and calculate the final consumer surplus
for sector in RegulatorETS.sector_registry.values():
    sector.free_emission_multiplier = best_multipliers[sector.name]
    
# Apply the scaling coefficient
try:
    coeff = RegulatorETS.exp7_find_coefficient_for_free_allocation(
        print_output=False, percentage=0.4
    )
    if coeff is None:
        raise ValueError("Scaling coefficient not found.")
    for sector in RegulatorETS.sector_registry.values():
        sector.free_emission_multiplier *= coeff
    
    # Run the optimization with the best multipliers
    RegulatorETS.optimization_with_positive_constraints_ab(
        print_output=True, gurobi_print=False
    )
    
    # Calculate and print the maximum consumer surplus
    consumer_surplus = sum(
        sector.get_consumer_surplus() for sector in RegulatorETS.sector_registry.values()
    )
    print(f"Maximum Consumer Surplus: {consumer_surplus}")
    
except Exception as e:
    print(f"An error occurred during the final simulation: {e}")

# old run 1,0,0,1,0.9,0

[I 2024-11-15 11:42:19,098] A new study created in memory with name: no-name-a8d82b18-2001-4f0c-ab61-9b8c5bc21e53


  0%|          | 0/100 [00:00<?, ?it/s]

No solution found
No solution found
No solution found
No solution found
No solution found
No solution found
No solution found
No solution found
No solution found
[I 2024-11-15 11:42:45,889] Trial 0 finished with value: -682872.1484526532 and parameters: {'multiplier_Steel': 0.45, 'multiplier_Cement': 0.7000000000000001, 'multiplier_Paper': 0.75, 'multiplier_Chemicals': 0.0, 'multiplier_Automotive': 0.5, 'multiplier_Textiles': 0.8}. Best is trial 0 with value: -682872.1484526532.
No solution found
No solution found
No solution found
No solution found
No solution found
No solution found
No solution found
No solution found
No solution found
[I 2024-11-15 11:43:12,057] Trial 1 finished with value: inf and parameters: {'multiplier_Steel': 0.6000000000000001, 'multiplier_Cement': 0.2, 'multiplier_Paper': 0.5, 'multiplier_Chemicals': 0.4, 'multiplier_Automotive': 0.9, 'multiplier_Textiles': 0.0}. Best is trial 0 with value: -682872.1484526532.
No solution found
No solution found
No solution f